In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
 

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/apoorvasharma/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/apoorvasharma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz

## Read Data

In [3]:
#loading dataset
# data = pd.read_table('amazon_reviews_us_Kitchen_v1_00.tsv')

df = pd.read_csv('amazon_reviews_us_Kitchen_v1_00.tsv', sep='\t', error_bad_lines=False)

b'Skipping line 16148: expected 15 fields, saw 22\nSkipping line 20100: expected 15 fields, saw 22\nSkipping line 45178: expected 15 fields, saw 22\nSkipping line 48700: expected 15 fields, saw 22\nSkipping line 63331: expected 15 fields, saw 22\n'
b'Skipping line 86053: expected 15 fields, saw 22\nSkipping line 88858: expected 15 fields, saw 22\nSkipping line 115017: expected 15 fields, saw 22\n'
b'Skipping line 137366: expected 15 fields, saw 22\nSkipping line 139110: expected 15 fields, saw 22\nSkipping line 165540: expected 15 fields, saw 22\nSkipping line 171813: expected 15 fields, saw 22\n'
b'Skipping line 203723: expected 15 fields, saw 22\nSkipping line 209366: expected 15 fields, saw 22\nSkipping line 211310: expected 15 fields, saw 22\nSkipping line 246351: expected 15 fields, saw 22\nSkipping line 252364: expected 15 fields, saw 22\n'
b'Skipping line 267003: expected 15 fields, saw 22\nSkipping line 268957: expected 15 fields, saw 22\nSkipping line 303336: expected 15 field

b'Skipping line 1968846: expected 15 fields, saw 22\nSkipping line 1999941: expected 15 fields, saw 22\nSkipping line 2001492: expected 15 fields, saw 22\nSkipping line 2011204: expected 15 fields, saw 22\nSkipping line 2025295: expected 15 fields, saw 22\n'
b'Skipping line 2041266: expected 15 fields, saw 22\nSkipping line 2073314: expected 15 fields, saw 22\nSkipping line 2080133: expected 15 fields, saw 22\nSkipping line 2088521: expected 15 fields, saw 22\n'
b'Skipping line 2103490: expected 15 fields, saw 22\nSkipping line 2115278: expected 15 fields, saw 22\nSkipping line 2153174: expected 15 fields, saw 22\nSkipping line 2161731: expected 15 fields, saw 22\n'
b'Skipping line 2165250: expected 15 fields, saw 22\nSkipping line 2175132: expected 15 fields, saw 22\nSkipping line 2206817: expected 15 fields, saw 22\nSkipping line 2215848: expected 15 fields, saw 22\nSkipping line 2223811: expected 15 fields, saw 22\n'
b'Skipping line 2257265: expected 15 fields, saw 22\nSkipping line

## Keep Reviews and Ratings

In [7]:
data = df[["review_id","review_body", "star_rating"]]
# print(data.info())
data=data.dropna(subset=['review_body'])
# print(data.info())

# Labelling Reviews:
## The reviews with rating 4,5 are labelled to be 1 and 1,2 are labelled as 0. Discard the reviews with rating 3'

In [8]:
discarded_count = len(data[data['star_rating'] == 3.0])
# print("Neutral Reviews: ", discarded_count)

Neutral Reviews:  349539


In [9]:
data.drop(data[data['star_rating'] == 3.0].index, inplace = True)
# print(data.info())

data['label'] = np.where(data['star_rating']>=4.0, 1, 0)
# data.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4525105 entries, 0 to 4874889
Data columns (total 3 columns):
 #   Column       Dtype  
---  ------       -----  
 0   review_id    object 
 1   review_body  object 
 2   star_rating  float64
dtypes: float64(1), object(2)
memory usage: 138.1+ MB
None


 ## We select 200000 reviews randomly with 100,000 positive and 100,000 negative reviews.



In [10]:
pos_data = data[data['label']==1]
neg_data = data[data['label']==0]

# print("Positive Reviews: ", len(pos_data))
# print("Negative Reviews: ", len(neg_data))

pos = pos_data.sample(n=100000, random_state=1)
neg = neg_data.sample(n=100000, random_state=1)

data_both = pd.concat([pos, neg], ignore_index=True)
data_both = data_both.sample(frac=1).reset_index(drop=True)
# data_both.head(10)

Positive Reviews:  3856296
Negative Reviews:  668809


In [12]:
from statistics import mean

data_both['review_body'] = data_both['review_body'].astype(str)

avg_len1 = mean([len(s) for s in data_both["review_body"]])

#before cleaing and pre-processing
# print("BEFORE CLEANING")
# print(*[rev+", length: "+str(len(rev))+"\n" for rev in data_both["review_body"].head(3)])
# print("Average review length before processing: "+ str(avg_len1))

BEFORE CLEANING
It is as pictured.  It is cool, unique gift idea.  I also purchased &#34;The Best Ice Mold&#34; (Amazon) and the round ice ball fits perfectly in this glass.  This was a birthday themed idea that I put together for my husband.  I also purchased  a bottle of Markers Mark whiskey (local) and a MM glass and coaster (Amazon).  He really liked it.  FYI: I also bought some real good cigars (locally) and a travel humidor (Amazon). to round out the whole theme....because lets face it, buying a surprise birthday gift for my husband of 30 years is a challenge LoL but a fun one.  I really scored with this theme. :-), length: 612
 The tags are satin finish and very hard to write on. Either the ink wipes off (sharpie) or is hard to write on (ball point)., length: 124
 This was really cheap made. It serves the purpose but tore very easily, length: 70

Average review length before processing: 323.274835


# Data Cleaning

In [13]:
#regexs for expressions to be removed
replace_url = re.compile(r'https?:\/\/.*[\r\n]*')
remove_href = re.compile(r'\<a href')
remove_amp = re.compile(r'&amp;') 
remove_sym = re.compile(r'[_"\-;%()|+&=*%.,~!?:#$\[\]/]')
remove_br = re.compile(r'<br />')
remove_space = re.compile(r' +')
replace_brackets = re.compile('[/(){}\[\]\|,;+_-]')
remove_numbers = re.compile("[\d]")


def cleaning_text(text):
    text = text.lower() # lowercase text
    text = replace_url.sub("", text) # delete URLs from text
    text = remove_numbers.sub("", text) # delete numbers tags from text
    text = remove_href.sub('', text) # delete HTML tags from text
    text = remove_amp.sub('', text) # delete "&" from text
    text = remove_sym.sub(' ', text)# replace "remove_sym" symbols by space in text
    text = remove_br.sub(' ', text) # delete HTML tags from text
    text = replace_brackets.sub(' ', text) # replace "replace_brackets" symbols by space in text
    text = remove_space.sub(' ', text)
    
    return text

#applying all the above cleaning to the dataset
data_both['review_body'] = data_both['review_body'].apply(cleaning_text)
# data_both.head(10)

,review_id,review_body,star_rating,label
0,RZMWGGF37ISTO,it is as pictured it is cool unique gift idea ...,4.0,1
1,R3BN73A6RT3LFP,the tags are satin finish and very hard to wri...,1.0,0
2,R1QRZ3W59ZMREX,this was really cheap made it serves the purpo...,2.0,0
3,R1T1KAXBW2HJEC,this product was shipped as a present to carry...,1.0,0
4,R1IP7DH28CEL72,unacceptable for my cooktop which requires tha...,1.0,0
5,R2W5RCO3L4VBHH,the only downside i see is that it takes up a ...,5.0,1
6,R1NIVXSOO7SH3J,i'm shocked this has good reviews i just got i...,2.0,0
7,R1RSNTGPQMLK2U,really like this pan perfect for large dishes ...,4.0,1
8,R3QFR9AAC4DGWQ,i purchased these when they were advertised as...,2.0,0
9,R256OUIH0I1QXW,quite large and a little cheap looking i have ...,2.0,0


## perform contractions on the reviews.

In [14]:
def contractionfunction(s):
    
    s = re.sub(r"won\'t", "will not", s)
    s = re.sub(r"can\'t", "can not", s)
    s = re.sub(r"ain\'t", "are not", s)
    s = re.sub(r"\'cause","because",s)
    s = re.sub(r"let\'s", "let us",s)
    s = re.sub(r"o\'clock", "of the clock", s)
    s = re.sub(r"ma\'am", "madam",s)
    
    s = re.sub(r"n\'t", " not", s)
    s = re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'d", " would", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    return s

data_both['review_body'] = data_both['review_body'].apply(contractionfunction)
# data_both.head(10)

,review_id,review_body,star_rating,label
0,RZMWGGF37ISTO,it is as pictured it is cool unique gift idea ...,4.0,1
1,R3BN73A6RT3LFP,the tags are satin finish and very hard to wri...,1.0,0
2,R1QRZ3W59ZMREX,this was really cheap made it serves the purpo...,2.0,0
3,R1T1KAXBW2HJEC,this product was shipped as a present to carry...,1.0,0
4,R1IP7DH28CEL72,unacceptable for my cooktop which requires tha...,1.0,0
5,R2W5RCO3L4VBHH,the only downside i see is that it takes up a ...,5.0,1
6,R1NIVXSOO7SH3J,i am shocked this has good reviews i just got ...,2.0,0
7,R1RSNTGPQMLK2U,really like this pan perfect for large dishes ...,4.0,1
8,R3QFR9AAC4DGWQ,i purchased these when they were advertised as...,2.0,0
9,R256OUIH0I1QXW,quite large and a little cheap looking i have ...,2.0,0


In [15]:
avg_len2 = mean([len(s) for s in data_both["review_body"]])

# #before cleaing and pre-processing
# print("AFTER CLEANING AND BEFORE PREPROCESSING")
# print(*[rev+", length: "+str(len(rev))+"\n" for rev in data_both["review_body"].head(3)])
# print("Average review length after cleaning and before processing: "+ str(avg_len2))

AFTER CLEANING AND BEFORE PREPROCESSING
it is as pictured it is cool unique gift idea i also purchased the best ice mold amazon and the round ice ball fits perfectly in this glass this was a birthday themed idea that i put together for my husband i also purchased a bottle of markers mark whiskey local and a mm glass and coaster amazon he really liked it fyi i also bought some real good cigars locally and a travel humidor amazon to round out the whole theme because lets face it buying a surprise birthday gift for my husband of years is a challenge lol but a fun one i really scored with this theme , length: 563
 the tags are satin finish and very hard to write on either the ink wipes off sharpie or is hard to write on ball point , length: 119
 this was really cheap made it serves the purpose but tore very easily, length: 69

Average review length after cleaning and before processing: 312.739695


# Pre-processing

## remove the stop words 

In [16]:
from nltk.corpus import stopwords
 
def remove_stopwords(text):
    STOPWORDS = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

data_both['review_body'] = data_both['review_body'].apply(remove_stopwords)
# data_both.head(10)

,review_id,review_body,star_rating,label
0,RZMWGGF37ISTO,pictured cool unique gift idea also purchased ...,4.0,1
1,R3BN73A6RT3LFP,tags satin finish hard write either ink wipes ...,1.0,0
2,R1QRZ3W59ZMREX,really cheap made serves purpose tore easily,2.0,0
3,R1T1KAXBW2HJEC,product shipped present carry overseas paid da...,1.0,0
4,R1IP7DH28CEL72,unacceptable cooktop requires cookware totally...,1.0,0
5,R2W5RCO3L4VBHH,downside see takes little room kitchen larger ...,5.0,1
6,R1NIVXSOO7SH3J,shocked good reviews got mail today excited us...,2.0,0
7,R1RSNTGPQMLK2U,really like pan perfect large dishes used calp...,4.0,1
8,R3QFR9AAC4DGWQ,purchased advertised popsicle holders pain kno...,2.0,0
9,R256OUIH0I1QXW,quite large little cheap looking seen much nic...,2.0,0


## perform tokenization and lemmatization  

In [17]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer

#creating an tokenizer object
text_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)

def tokenize_text(text):
    #tokenize text
    review_tokens = text_tokenizer.tokenize(text)
    
    return review_tokens
    
#tokenizing each review in the dataset
data_both['review_body'] = data_both['review_body'].apply(tokenize_text)
# data_both.head(10)

,review_id,review_body,star_rating,label
0,RZMWGGF37ISTO,"[pictured, cool, unique, gift, idea, also, pur...",4.0,1
1,R3BN73A6RT3LFP,"[tags, satin, finish, hard, write, either, ink...",1.0,0
2,R1QRZ3W59ZMREX,"[really, cheap, made, serves, purpose, tore, e...",2.0,0
3,R1T1KAXBW2HJEC,"[product, shipped, present, carry, overseas, p...",1.0,0
4,R1IP7DH28CEL72,"[unacceptable, cooktop, requires, cookware, to...",1.0,0
5,R2W5RCO3L4VBHH,"[downside, see, takes, little, room, kitchen, ...",5.0,1
6,R1NIVXSOO7SH3J,"[shocked, good, reviews, got, mail, today, exc...",2.0,0
7,R1RSNTGPQMLK2U,"[really, like, pan, perfect, large, dishes, us...",4.0,1
8,R3QFR9AAC4DGWQ,"[purchased, advertised, popsicle, holders, pai...",2.0,0
9,R256OUIH0I1QXW,"[quite, large, little, cheap, looking, seen, m...",2.0,0


In [18]:
#stemmer object
stemmer=PorterStemmer()

#Lemmatizer object 
lemmmatizer=WordNetLemmatizer()

def lemmatize_tokens(tokens):
    #stemming
    stemmed_tokens=[stemmer.stem(t) for t in tokens]

    #Stemming with lemmatization
    lemmatized_tokens = [lemmmatizer.lemmatize(st) for st in stemmed_tokens]
    lemmatize_review = ' '.join(lemmatized_tokens)
    
    return lemmatize_review
    
data_both['review_body'] = data_both['review_body'].apply(lemmatize_tokens)

In [19]:
from statistics import mean

avg_len3 = mean([len(s) for s in data_both["review_body"]])

# #after cleaing and pre-processing
# print("AFTER PREPROCESSING")
# print(*[rev+", length: "+str(len(rev))+"\n" for rev in data_both["review_body"].head(3)])
# print("Average review lentgh after processing: "+ str(avg_len3))

AFTER PREPROCESSING
pictur cool uniqu gift idea also purchas best ice mold amazon round ice ball fit perfectli glass birthday theme idea put togeth husband also purchas bottl marker mark whiskey local mm glass coaster amazon realli like fyi also bought real good cigar local travel humidor amazon round whole theme let face buy surpris birthday gift husband year challeng lol fun one realli score theme, length: 382
 tag satin finish hard write either ink wipe sharpi hard write ball point, length: 72
 realli cheap made serv purpos tore easili, length: 41

Average review lentgh after processing: 176.841055


In [20]:
from sklearn.model_selection import train_test_split

#splitting the dataset into train and test sets
X = data_both.review_body
y = data_both.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Perceptron

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import Perceptron

#perceptron for reviews
perceptron = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', Perceptron(tol=1e-3, random_state=0)),
               ])
#training the model
perceptron.fit(X_train, y_train)

# print("---PERCEPTRON---")
#predicting the classes on train
y_pred_perceptron_train = perceptron.predict(X_train)

# #metrics
# print('Train Accuracy: %s' % accuracy_score(y_pred_perceptron_train, y_train))
# print('Train Precision: %s' % precision_score(y_pred_perceptron_train, y_train))
# print('Train Recall: %s' % recall_score(y_pred_perceptron_train, y_train))
# print('Train F1: %s \n' % f1_score(y_pred_perceptron_train, y_train))

#predicting the classes on test
y_pred_perceptron = perceptron.predict(X_test)

#metrics
# print('Test Accuracy: %s' % accuracy_score(y_pred_perceptron, y_test))
# print('Test Precision: %s' % precision_score(y_pred_perceptron, y_test))
# print('Test Recall: %s' % recall_score(y_pred_perceptron, y_test))
# print('Test F1: %s' % f1_score(y_pred_perceptron, y_test))

---PERCEPTRON---
Train Accuracy: 0.88214375
Train Precision: 0.910613269396471
Train Recall: 0.8617466320018908
Train F1: 0.885506287227002 

Test Accuracy: 0.84735
Test Precision: 0.8781185683449626
Test Recall: 0.8262717868782768
Test F1: 0.8514065998247835


# SVM

In [23]:
#SVM classifier for reviews
from sklearn.linear_model import SGDClassifier

svm_classifier = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-5, max_iter=100, tol=None)),
               ])
#training the model
svm_classifier.fit(X_train, y_train)

# print("---SVM---")
#predicting the classes on train
y_pred_svm_train = svm_classifier.predict(X_train)

#metrics
# print('Train Accuracy: %s' % accuracy_score(y_pred_svm_train, y_train))
# print('Train Precision: %s' % precision_score(y_pred_svm_train, y_train))
# print('Train Recall: %s' % recall_score(y_pred_svm_train, y_train))
# print('Train F1: %s \n' % f1_score(y_pred_svm_train, y_train))

#predicting the classes on test
y_pred_svm = svm_classifier.predict(X_test)

#metrics
# print('Test Accuracy: %s' % accuracy_score(y_pred_svm, y_test))
# print('Test Precision: %s' % precision_score(y_pred_svm, y_test))
# print('Test Recall: %s' % recall_score(y_pred_svm, y_test))
# print('Test F1: %s' % f1_score(y_pred_svm, y_test))

---SVM---
Train Accuracy: 0.91041875
Train Precision: 0.9053559609885238
Train Recall: 0.9147803265450324
Train F1: 0.9100437448613908 

Test Accuracy: 0.893275
Test Precision: 0.8891119923698609
Test Recall: 0.8958122597612785
Test F1: 0.8924495502985413


# Logistic Regression

In [24]:
#logistic regression model for reviews
from sklearn.linear_model import LogisticRegression

log_reg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=5, C=1e6)),
               ])
#training the model
log_reg.fit(X_train, y_train)

# print("---LR---")

#predicting the classes on train
y_pred_lr_train = log_reg.predict(X_train)

#metrics
# print('Train Accuracy: %s' % accuracy_score(y_pred_lr_train, y_train))
# print('Train Precision: %s' % precision_score(y_pred_lr_train, y_train))
# print('Train Recall: %s' % recall_score(y_pred_lr_train, y_train))
# print('Train F1: %s \n' % f1_score(y_pred_lr_train, y_train))

#predicting the classes on test
y_pred_lr = log_reg.predict(X_test)

#metrics
# print('Test Accuracy: %s' % accuracy_score(y_pred_lr, y_test))
# print('Test Precision: %s' % precision_score(y_pred_lr, y_test))
# print('Test Recall: %s' % recall_score(y_pred_lr, y_test))
# print('Test F1: %s' % f1_score(y_pred_lr, y_test))

---LR---
Train Accuracy: 0.9239125
Train Precision: 0.9226264064236567
Train Recall: 0.9251449393320895
Train F1: 0.9238839564836815 

Test Accuracy: 0.881875
Test Precision: 0.8807790773555544
Test Recall: 0.8818858061921995
Test F1: 0.8813320943315669


# Naive Bayes

In [25]:
#Multinomial Naive Bayes Classifier for reviews
from sklearn.naive_bayes import MultinomialNB

multi_naive_bayes = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
#training the classifier
multi_naive_bayes.fit(X_train, y_train)

# print("---NB---")

#predicting the classes on train
y_pred_nb_train = multi_naive_bayes.predict(X_train)

#metrics
# print('Train Accuracy: %s' % accuracy_score(y_pred_nb_train, y_train))
# print('Train Precision: %s' % precision_score(y_pred_nb_train, y_train))
# print('Train Recall: %s' % recall_score(y_pred_nb_train, y_train))
# print('Train F1: %s \n' % f1_score(y_pred_nb_train, y_train))

#predicting the classes on test
y_pred_nb = multi_naive_bayes.predict(X_test)

#metrics
# print('Test Accuracy: %s' % accuracy_score(y_pred_nb, y_test))
# print('Test Precision: %s' % precision_score(y_pred_nb, y_test))
# print('Test Recall: %s' % recall_score(y_pred_nb, y_test))
# print('Test F1: %s' % f1_score(y_pred_nb, y_test))


---NB---
Train Accuracy: 0.87893125
Train Precision: 0.8714269658712022
Train Recall: 0.884919729133379
Train F1: 0.8781215198474868 

Test Accuracy: 0.867075
Test Precision: 0.8590934190050701
Test Recall: 0.8720953933958419
Test F1: 0.8655455809836895


In [37]:
#final output

print("\nNeutral Reviews:", discarded_count, "Positive Reviews:", len(pos_data), "Negative Reviews:", len(neg_data), "\n")

print(str(avg_len1)+","+ str(avg_len2))
print(str(avg_len2)+","+ str(avg_len3)+"\n")

print(str(accuracy_score(y_pred_perceptron_train, y_train))+","+str(precision_score(y_pred_perceptron_train, y_train))+","+ str(recall_score(y_pred_perceptron_train, y_train))+","+str(f1_score(y_pred_perceptron_train, y_train))+","+str(accuracy_score(y_pred_perceptron, y_test))+","+str(precision_score(y_pred_perceptron, y_test))+","+ str(recall_score(y_pred_perceptron, y_test))+","+str(f1_score(y_pred_perceptron, y_test))+"\n")
print(str(accuracy_score(y_pred_svm_train, y_train))+","+str(precision_score(y_pred_svm_train, y_train))+","+ str(recall_score(y_pred_svm_train, y_train))+","+str(f1_score(y_pred_svm_train, y_train))+","+str(accuracy_score(y_pred_svm, y_test))+","+str(precision_score(y_pred_svm, y_test))+","+ str(recall_score(y_pred_svm, y_test))+","+str(f1_score(y_pred_svm, y_test))+"\n")
print(str(accuracy_score(y_pred_lr_train, y_train))+","+str(precision_score(y_pred_lr_train, y_train))+","+ str(recall_score(y_pred_lr_train, y_train))+","+str(f1_score(y_pred_lr_train, y_train))+","+str(accuracy_score(y_pred_lr, y_test))+","+str(precision_score(y_pred_lr, y_test))+","+ str(recall_score(y_pred_lr, y_test))+","+str(f1_score(y_pred_lr, y_test))+"\n")
print(str(accuracy_score(y_pred_nb_train, y_train))+","+str(precision_score(y_pred_nb_train, y_train))+","+ str(recall_score(y_pred_nb_train, y_train))+","+str(f1_score(y_pred_nb_train, y_train))+","+str(accuracy_score(y_pred_nb, y_test))+","+str(precision_score(y_pred_nb, y_test))+","+ str(recall_score(y_pred_nb, y_test))+","+str(f1_score(y_pred_nb, y_test))+"\n")

Neutral Reviews: 349539 Positive Reviews: 3856296 Negative Reviews: 668809 

323.274835,312.739695
312.739695,176.841055

0.88214375,0.910613269396471,0.8617466320018908,0.885506287227002,0.84735,0.8781185683449626,0.8262717868782768,0.8514065998247835

0.91041875,0.9053559609885238,0.9147803265450324,0.9100437448613908,0.893275,0.8891119923698609,0.8958122597612785,0.8924495502985413

0.9239125,0.9226264064236567,0.9251449393320895,0.9238839564836815,0.881875,0.8807790773555544,0.8818858061921995,0.8813320943315669

0.87893125,0.8714269658712022,0.884919729133379,0.8781215198474868,0.867075,0.8590934190050701,0.8720953933958419,0.8655455809836895

